In [1]:
import os
import pandas as pd
import subprocess
from shapely.geometry import Point
import geopandas as gpd

## Zenodo
Download csv with counting points from Zenodo, using an R script. The R script currently contains a placeholder file.

In [2]:
#-- Download
env = os.environ.copy()
env['R_LIBS'] = 'C:/R/library' # Asjust to folder where R libraries are installed

# Download directory
download_dir = os.path.join(os.getcwd(),'..','data','raw')

command = [
    "C:/R/R-4.4.1/bin/Rscript", # Asjust to version of R you want to run
    "func_telpunten_zenodo.R",
    download_dir
]

result = subprocess.run(command, capture_output=True, text=True, env=env)

print("STDOUT:", result.stdout)
print("STDERR:", result.stderr)


STDOUT: [1] "File was downloaded"

STDERR: Will download 1 file (total size: 9.4 KiB) from https://doi.org/10.5281/zenodo.10890741 (Beaver territory locations in Flanders 2014-2019; version: 01/01/2020)


 Downloaded 15 bytes...
 Downloaded 1508 bytes...
 Downloaded 3510 bytes...
Verifying file integrity...

Beaver_territories.csv was downloaded and its integrity verified (md5sum: 2cd5b6bec7998f44d94136b6421cf8e7)



## Reading geometries
First convert csv with counting points to geodataferame with counting circles (r=300m).
Then clip the parcel data with this geodataframe.

In [3]:
# Read the downloaded csv file with counting points
df_pts = pd.read_csv('../data/raw/steekproef_mbag_mas.csv')

# create GeoDataFrame
# Step 1: Create a 'geometry' column with Point geometries
df_pts['geometry'] = df_pts.apply(lambda row: Point(row['x_coord'], row['y_coord']), axis=1)
df_pts_crs = df_pts.crs[0]
df_pts = df_pts[['pointid', 'geometry']]

# Step 2: Convert the DataFrame to a GeoDataFrame
tc = gpd.GeoDataFrame(df_pts, geometry='geometry')

# Step 3: Set a coordinate reference system (CRS) (units should be in meters!!)
tc = tc.set_crs(epsg=int(df_pts_crs), inplace=True)

# Step 4: Buffer 300
tc = tc.buffer(300)


# Now df_pts is a GeoDataFrame with the original 'id' column
tc.plot()

NameError: name 'Point' is not defined

In [ ]:
tc.to_file('../data/processed/telpunten_processed.gpkg')